## T20 World Cup 2022 Data Collection using Web Scraping

### Importing necessary libraries

In [1]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs

### Scrapping match results from espncricinfo

In [2]:
r = req.get("https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament")
soup = bs(r.content)
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/CiIcons/ci-icons-v2.15/fonts/icomoon.woff?gencn3" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/BentonSans/BentonSans-Bold/BentonSans-Bold.woff" rel="preload" type="font/woff"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/BentonSans/BentonSans-Regular/BentonSans-Regular.woff" rel="preload" type="font/woff"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/BentonSans/BentonSans-Medium/BentonSans-Medium.woff" rel="preload" type="font/woff"/>
  <link crossorigin="anonymous" href="https://wassets.hscicdn.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://img1.hscicdn.com" rel="preconnect"/>
  <script type="text/javascript">
   /*! js-cookie v3.0.5 | MIT */
 

In [4]:
values = []
info = soup.select("table tr")
for i, r in enumerate(info):
    data = r.select('td')
    x = []
    for a in data:
        x.append(a.get_text())
    values.append(x)
    

results = pd.DataFrame(values[1:], columns = values[0])
# Saving match result data as CSV
results.to_csv("Match results.csv", index = False)

<td class="ds-min-w-max">
 <div class="ds-popper-wrapper">
  <span class="ds-cursor-pointer">
   Team 1
  </span>
 </div>
</td>

<td class="ds-min-w-max ds-text-right">
 <div class="ds-popper-wrapper">
  <span class="ds-cursor-pointer">
   Team 2
  </span>
 </div>
</td>

<td class="ds-min-w-max ds-text-right">
 <div class="ds-popper-wrapper">
  <span class="ds-cursor-pointer">
   Winner
  </span>
 </div>
</td>

<td class="ds-min-w-max ds-text-right">
 <div class="ds-popper-wrapper">
  <span class="ds-cursor-pointer">
   Margin
  </span>
 </div>
</td>

<td class="ds-min-w-max ds-text-right">
 <div class="ds-popper-wrapper">
  <span class="ds-cursor-pointer">
   Ground
  </span>
 </div>
</td>

<td class="ds-min-w-max ds-text-right ds-whitespace-nowrap">
 <div class="ds-popper-wrapper">
  <span class="ds-cursor-pointer">
   Match Date
  </span>
 </div>
</td>

<td class="ds-min-w-max ds-text-right">
 <div class="ds-popper-wrapper">
  <span class="ds-cursor-pointer">
   Scorecard
  </span>


### Scrapping batting and bowling stats

In [5]:
# Getting scorecard links of each match
x = soup.select("table tbody tr")
links = []
for i, r in enumerate(x):
    data = r.select("td a")[-1]
    links.append(data['href'])

print(links[40])

/series/icc-men-s-t20-world-cup-2022-23-1298134/england-vs-india-2nd-semi-final-1298178/full-scorecard


#### Batting stats

In [5]:
#Extarcting batting stats
path = 'https://www.espncricinfo.com'
url = path+links[40]
t = req.get(url)
test = bs(t.content)
print(test.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/CiIcons/ci-icons-v2.15/fonts/icomoon.woff?gencn3" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/BentonSans/BentonSans-Bold/BentonSans-Bold.woff" rel="preload" type="font/woff"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/BentonSans/BentonSans-Regular/BentonSans-Regular.woff" rel="preload" type="font/woff"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/BentonSans/BentonSans-Medium/BentonSans-Medium.woff" rel="preload" type="font/woff"/>
  <link crossorigin="anonymous" href="https://wassets.hscicdn.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://img1.hscicdn.com" rel="preconnect"/>
  <script type="text/javascript">
   /*! js-cookie v3.0.5 | MIT */
 

In [86]:
def batting_stat(innings, team, match, l = 0):
    res = []
    x = 0
    for c, i in enumerate(innings):
        if x == -1:
            break
        data = i.select("td")
        temp_dict = {'Match':match, 'Team':team}
        for n, j in enumerate(data):
            if n == 0:
                #data = j.find("span")
                if j.get_text().replace('\xa0', '') == 'Extras':
                    temp_dict = None
                    x = -1
                    break
                elif j.get_text().replace('\xa0', '') == '':
                    temp_dict = None
                    continue
                else:
                    l = l + 1
                    temp_dict['BattingPos'] = l
                    temp_dict['Batsman_name'] = j.get_text().replace('\xa0', '')
            elif n == 1:
                temp_dict['Dismissal'] = j.get_text().replace("\xa0", '')
            elif n == 2:
                #data = j.find("strong")
                temp_dict['Runs'] = j.get_text().replace('\xa0', '')
            elif n == 3:
                temp_dict['Balls'] = j.get_text().replace('\xa0', '')
            elif n == 4:
                continue
            elif n == 5:
                temp_dict['Fours'] = j.get_text().replace('\xa0', '')
            elif n == 6:
                temp_dict['Sixes'] = j.get_text().replace('\xa0', '')
            else:
                temp_dict['SR'] = j.get_text().replace('\xa0', '')

        if temp_dict != None: 
            #return temp_dict
            res.append(temp_dict)
        else:
            continue
            
    return res

In [30]:
def batting(url):
    t = req.get(url)
    content = bs(t.content)
    tables = content.find_all("table", attrs={"class":"ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table"})
    first = tables[0]
    second = tables[1]
    ran = content.select("div.ds-flex.ds-flex-row.ds-w-full.ds-overflow-x-auto.ds-scrollbar-hide span.ds-text-tight-xs")
    t1 = ran[0].get_text().replace(" Innings", "")
    t2 = ran[1].get_text().replace(" Innings", "")
    match_info = t1 +" Vs "+ t2
    temp = []
    # Getting 1st innings batting stats
    firstInnings = first.find("tbody").select('tr')
    # Getting 2nd innings batting stats
    secondInnings = second.find('tbody').select('tr')
    temp.append(batting_stat(firstInnings, t1, match_info))
    temp.append(batting_stat(secondInnings, t2, match_info))
    return temp

In [94]:
# Extracting battingstats of every match from Worldcup 2022
battingstats = {}
for i, j in enumerate(links):

    path = 'https://www.espncricinfo.com'
    url = path+j
    try:
        key = 'Match #'+str(i+1)
        battingstats[key] = batting(url)
        #battingstats.append(batting(url))
    except Exception as e:
        print(e)
        print(j)
    

battingstats
    

{'Match #1': [[{'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Namibia',
    'BattingPos': 1,
    'Batsman_name': 'Michael van Lingen',
    'Dismissal': 'c Pramod Madushan b Chameera',
    'Runs': '3',
    'Balls': '6',
    'Fours': '0',
    'Sixes': '0',
    'SR': '50.00'},
   {'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Namibia',
    'BattingPos': 2,
    'Batsman_name': 'Divan la Cock',
    'Dismissal': 'c Shanaka b Pramod Madushan',
    'Runs': '9',
    'Balls': '9',
    'Fours': '1',
    'Sixes': '0',
    'SR': '100.00'},
   {'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Namibia',
    'BattingPos': 3,
    'Batsman_name': 'Jan Nicol Loftie-Eaton',
    'Dismissal': 'c †Mendis b Karunaratne',
    'Runs': '20',
    'Balls': '12',
    'Fours': '1',
    'Sixes': '2',
    'SR': '166.66'},
   {'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Namibia',
    'BattingPos': 4,
    'Batsman_name': 'Stephan Baard',
    'Dismissal': 'c DM de Silva b Pramod Madushan',
    'Runs': '26',
    'Balls': '

#### Saving batting stats as JSON

In [22]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [98]:
save_data("t20_wc_battingsummary", battingstats)

#### Bowling stats

In [33]:
def bowling_stats(innings, team, match):
    arr = []
    for i, j in enumerate(innings):
        if j.find("div", {'class':'ds-text-tight-s ds-font-regular'}):
            continue
        else:
            bowl_data = j.select("td")
            bowl_dict = {'Match':match, 'Team':team}
            for m, n in enumerate(bowl_data):
                if m == 0:
                    bowl_dict['Bowler_Name'] = n.get_text().replace('\xa0', '')
                elif m == 1:
                    bowl_dict['Overs'] = n.get_text().replace('\xa0', '')
                elif m == 2:
                    bowl_dict['Maidens'] = n.get_text().replace('\xa0', '')
                elif m == 3:
                    bowl_dict['Runs'] = n.get_text().replace('\xa0', '')
                elif m == 4:
                    bowl_dict['Wickets'] = n.get_text().replace('\xa0', '')
                elif m == 5:
                    bowl_dict['Economy'] = n.get_text().replace('\xa0', '')
                elif m == 6:
                    bowl_dict['Dots'] = n.get_text().replace('\xa0', '')
                elif m == 7:
                    bowl_dict['Fours'] = n.get_text().replace('\xa0', '')
                elif m == 8:
                    bowl_dict['Sixes'] = n.get_text().replace('\xa0', '')
                elif m == 9:
                    bowl_dict['Wides'] = n.get_text().replace('\xa0', '')
                else:
                    bowl_dict['No balls'] = n.get_text().replace('\xa0', '')

            arr.append(bowl_dict)

    return arr


In [34]:
def bowling(url):
    t = req.get(url)
    bowl = bs(t.content)
    bowl_tables = bowl.find_all("table", attrs= {'class':'ds-w-full ds-table ds-table-md ds-table-auto'})
    first_bowl = bowl_tables[0]
    second_bowl = bowl_tables[1]
    names = bowl.select("div.ds-flex.ds-flex-row.ds-w-full.ds-overflow-x-auto.ds-scrollbar-hide span.ds-text-tight-xs")
    team1 = names[0].get_text().replace(" Innings", "")
    team2 = names[1].get_text().replace(" Innings", "")
    match = team1 +" Vs "+ team2
    temp_arr = []
    first_bowling = first_bowl.find('tbody').select('tr')
    second_bowling = second_bowl.find("tbody").select("tr")
    temp_arr.append(bowling_stats(first_bowling, team2, match))
    temp_arr.append(bowling_stats(second_bowling, team1, match))
    return temp_arr

In [35]:
# Extracting bowlingstats of every match from Worldcup 2022
bowlingstats = {}
for i, j in enumerate(links):

    path = 'https://www.espncricinfo.com'
    url = path+j
    try:
        key = 'Match #'+str(i+1)
        bowlingstats[key] = bowling(url)
        
    except Exception as e:
        print(e)
        print(j)
    

bowlingstats
    

{'Match #1': [[{'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Sri Lanka',
    'Bowler_Name': 'Maheesh Theekshana',
    'Overs': '4',
    'Maidens': '0',
    'Runs': '23',
    'Wickets': '1',
    'Economy': '5.75',
    'Dots': '7',
    'Fours': '0',
    'Sixes': '0',
    'Wides': '2',
    'No balls': '0'},
   {'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Sri Lanka',
    'Bowler_Name': 'Dushmantha Chameera',
    'Overs': '4',
    'Maidens': '0',
    'Runs': '39',
    'Wickets': '1',
    'Economy': '9.75',
    'Dots': '6',
    'Fours': '3',
    'Sixes': '1',
    'Wides': '2',
    'No balls': '0'},
   {'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Sri Lanka',
    'Bowler_Name': 'Pramod Madushan',
    'Overs': '4',
    'Maidens': '0',
    'Runs': '37',
    'Wickets': '2',
    'Economy': '9.25',
    'Dots': '6',
    'Fours': '3',
    'Sixes': '1',
    'Wides': '0',
    'No balls': '0'},
   {'Match': 'Namibia Vs Sri Lanka',
    'Team': 'Sri Lanka',
    'Bowler_Name': 'Chamika Karunaratne',
 

In [38]:
#saving bowling stats as JSON
save_data("t20_wc_bowlingsummary", bowlingstats)

### Extarcting Player Info

In [84]:
def playerlinks(url):
    p = req.get(url)
    soup = bs(p.content)
    batting_table = soup.find_all('table', {'class':'ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table'})
    bowling_table = soup.find_all('table', {'class':'ds-w-full ds-table ds-table-md ds-table-auto'})
    overall_tables = [batting_table, bowling_table]
    player_links = []
    player_names = []
    base_path = 'https://www.espncricinfo.com'
    for table in overall_tables:
        for i in table:
            j = i.find("tbody").select("tr")
            for r, row in enumerate(j):
                if row.select("td div strong"):
                    continue
                else:
                    temp = row.select("td a")
                    if temp != []:
                        x = temp[0]
                        name = x.get_text().replace("\xa0", "")
                        #print(name)
                        if name not in player_names:
                            player_names.append(name)
                            full_path = base_path + x['href']
                            player_links.append(full_path)
                        else:
                            continue

                    else:
                        continue
                        
    return player_links

In [92]:
# Extracting player info of all the players that played in wc2022
players = []
for i, j in enumerate(links):

    path = 'https://www.espncricinfo.com'
    url = path+j
    try:
        players_match = playerlinks(url)
        for k in players_match:
            players.append(k)
        
    except Exception as e:
        print(e)
        print(j)
    

players
    

['https://www.espncricinfo.com/cricketers/michael-van-lingen-833777',
 'https://www.espncricinfo.com/cricketers/divan-la-cock-1178065',
 'https://www.espncricinfo.com/cricketers/jan-nicol-loftie-eaton-1071473',
 'https://www.espncricinfo.com/cricketers/stephan-baard-437927',
 'https://www.espncricinfo.com/cricketers/gerhard-erasmus-519070',
 'https://www.espncricinfo.com/cricketers/jan-frylinck-482478',
 'https://www.espncricinfo.com/cricketers/david-wiese-221140',
 'https://www.espncricinfo.com/cricketers/jj-smit-553821',
 'https://www.espncricinfo.com/cricketers/pathum-nissanka-1028655',
 'https://www.espncricinfo.com/cricketers/kusal-mendis-629074',
 'https://www.espncricinfo.com/cricketers/dhananjaya-de-silva-465793',
 'https://www.espncricinfo.com/cricketers/danushka-gunathilaka-345821',
 'https://www.espncricinfo.com/cricketers/bhanuka-rajapaksa-342619',
 'https://www.espncricinfo.com/cricketers/dasun-shanaka-437316',
 'https://www.espncricinfo.com/cricketers/wanindu-hasaranga-78

In [93]:
#Removing duplicates
distinct_players = []
[distinct_players.append(i) for i in players if i not in distinct_players]

#saviing playerlinks as JSON
save_data("Player_links", distinct_players)

In [27]:
def info(url):
    pin = req.get(url)
    test = bs(pin.content)
    info = test.find("div", {'class':'ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8'})
    team = test.find("div", {'class':'ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-y-4'}).find('a')
    name = test.find("div", {'class':'ds-bg-raw-black/85'}).find("h1")
    info_dict = {}
    info_dict['Name'] = name.get_text()
    for n in info:
        key = n.find('p').get_text()
        if key == 'Born':
            continue
        else:
            info_dict[key] = n.find('span').get_text()
        
        

    info_dict['Team'] = team.get_text()
    return info_dict

In [28]:
import json
with open("Player_links", encoding='utf-8') as f:
    players = json.load(f)

player_info = []
for i, p in enumerate(players):

    try:
        player_info.append(info(p))
    except Exception as e:
        print(e)
        print(p)

player_info
    

[{'Name': 'Michael van Lingen',
  'Full Name': 'Michael van Lingen',
  'Age': '25y 213d',
  'Batting Style': 'Left hand Bat',
  'Bowling Style': 'Left arm Medium, Slow Left arm Orthodox',
  'Playing Role': 'Bowling Allrounder',
  'Team': 'Namibia'},
 {'Name': 'Divan la Cock',
  'Full Name': 'Divan la Cock',
  'Age': '20y 91d',
  'Batting Style': 'Right hand Bat',
  'Bowling Style': 'Legbreak',
  'Playing Role': 'Opening Batter',
  'Team': 'Namibia'},
 {'Name': 'Jan Nicol Loftie-Eaton',
  'Full Name': 'Jan Nicol Loftie-Eaton',
  'Age': '22y 71d',
  'Batting Style': 'Left hand Bat',
  'Bowling Style': 'Right arm Medium, Legbreak',
  'Playing Role': 'Batter',
  'Education': 'Paul Roos Gymnansium',
  'Team': 'Namibia'},
 {'Name': 'Stephan Baard',
  'Full Name': 'Stephan Julian Baard',
  'Age': '31y 26d',
  'Batting Style': 'Right hand Bat',
  'Bowling Style': 'Right arm Medium fast',
  'Playing Role': 'Batter',
  'Team': 'Namibia'},
 {'Name': 'Gerhard Erasmus',
  'Full Name': 'Merwe Gerhar

In [29]:
## Saving player info as JSON
save_data("t20_wc_player_info", player_info)
